# $k$-means-based approximate spectral clustering

This algorithm is from "[Fast Approximate Spectral Clustering](https://people.eecs.berkeley.edu/~jordan/papers/yan-huang-jordan-kdd09.pdf)" by Yan et el.

In [1]:
import numpy as np
import scipy as sp
import scipy.sparse.linalg
import scipy.cluster.vq

In [2]:
A = np.loadtxt('../data/processed/usps.csv', delimiter=',')

In [3]:
inds = A[:, -1] < 3
X = A[inds, :-2]
Y = A[inds, -1].astype(int)
k = len(np.unique(Y))
n, d = X.shape
n, d

(2199, 255)

In [4]:
# data reduction ratio
gamma = 2
k_prime = n // gamma

In [51]:
centroids_prime, distortion_prime = sp.cluster.vq.kmeans(X, k_prime)
distortion_prime

1.3720306925894707

In [52]:
y_prime = np.empty(n)
for i in range(n):
    dists = np.array([np.linalg.norm(X[i] - c) for c in centroids_prime])
    y_prime[i] = np.argmin(dists)

In [144]:
X_prime = centroids_prime
n_prime, d_prime = X_prime.shape
n_prime, d_prime

(873, 255)

In [174]:
W = np.empty((n_prime, n_prime))
for i in range(n_prime):
    for j in range(i, n_prime):
#         val = np.e ** (-1 * np.linalg.norm(X_prime[i] - X_prime[j]) ** 2)
        val = np.linalg.norm(X_prime[i] - X_prime[j]) ** 2
        W[i, j] = val
        W[j, i] = val

In [175]:
ww = W.sum(axis=0)
D = np.diag(ww)
D_ = np.diag(1 / np.sqrt(ww))
L = np.identity(n_prime) - D_.dot(W).dot(D_)

In [187]:
V, Z = sp.linalg.eigh(L, eigvals=(n_prime-2, n_prime-1))

In [188]:
Z_ = sp.cluster.vq.whiten(Z)
centroids, distortion = sp.cluster.vq.kmeans(Z_, k)
centroids, distortion

(array([[ 0.04305662, -1.11852681],
        [-0.04315428,  0.80547993]]), 0.83118902565668784)

In [189]:
y_hat_prime = np.empty(n_prime)
for i in range(n_prime):
    dists = np.array([np.linalg.norm(Z_[i] - c) for c in centroids])
    y_hat_prime[i] = np.argmin(dists)

In [190]:
y_hat = np.empty(n)
for i in range(n):
    y_hat[i] = y_hat_prime[ int(y_prime[i]) ] + 1

In [191]:
accuracy = np.empty(k)
for i in range(k):
    y_hat = (y_hat + i) % (k + 1)
    y_hat[y_hat == 0] = 1
    accuracy[i] = (Y == y_hat).sum() / n * 100
accuracy.max()

99.045020463847194